# NVIDIA SSD Evaluation fix
For our purpose, we want only EVAL

In [1]:
import os
import time
from argparse import ArgumentParser
import torch
import numpy as np
from torch.optim.lr_scheduler import MultiStepLR
import torch.utils.data.distributed

from src.model import SSD300, ResNet, Loss
from src.utils import dboxes300_coco, Encoder
from src.logger import Logger, BenchLogger
from src.evaluate import evaluate
# from src.train import train_loop, tencent_trick, load_checkpoint, benchmark_train_loop, benchmark_inference_loop
from src.train import train_loop, tencent_trick, load_checkpoint, benchmark_train_loop, benchmark_inference_loop
# from src.data import get_train_loader, get_val_dataset, get_val_dataloader, get_coco_ground_truth
from src.data import get_val_dataset, get_val_dataloader, get_coco_ground_truth


In [2]:
from src.main import train
from argparse import ArgumentParser

args = ArgumentParser().parse_args(args=[])
args.backbone = "resnet50"
args.mode = "evaluation"
args.checkpoint = "'./SSD_orig_state.st'"
args.data = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/coco"
args.no_cuda = False
args.seed = None
args.batch_size = 1
args.eval_batch_size = 1
args.num_workers = 1
args.backbone_path = None
args.learning_rate = 2.6e-3
args.momentum = 0.9
args.weight_decay = 0.0005
args.multistep = [43, 54]
# args.amp = 'store_true'
args.amp = None

# train(train_loop, Logger('Training logger', print_freq=1), args)


In [3]:
args.no_cuda = False
args.amp = None
args.distributed = False
args.local_rank = 0
args.backbone_path = None
args.checkpoint = "/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/SSD_new.ckpt"

ssd300 = SSD300(backbone=ResNet(args.backbone, args.backbone_path))
ssd300.cuda()
load_checkpoint(ssd300.module if args.distributed else ssd300, args.checkpoint)
checkpoint = torch.load(args.checkpoint,
                                    map_location=lambda storage, loc: storage.cuda(torch.cuda.current_device()))

val_dataset = get_val_dataset(args)

split1, _ = torch.utils.data.random_split(val_dataset, [200, len(val_dataset)-200])

val_dataloader = get_val_dataloader(split1, args)

cocoGt = get_coco_ground_truth(args)

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

inv_map = {v: k for k, v in val_dataset.label_map.items()}

ssd300.load_state_dict(torch.load("./SSD_new.ckpt"))
            
# acc = evaluate(ssd300, val_dataloader, cocoGt, encoder, inv_map, args)

loading model checkpoint /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/SSD/SSD_new.ckpt
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!


<All keys matched successfully>

In [4]:
import torch 
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
torch.save(ssd_model.state_dict(), './SSD_new.ckpt')

Using cache found in /home/bohumil/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [17]:
ssd_model

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [5]:
evaluate(ssd_model, val_dataloader, cocoGt, encoder, inv_map, args)


Parsing batch: 199/200
Predicting Ended, total time: 141.04 s
Loading and preparing results...
Converting ndarray to lists...
(21248, 7)
0/21248
DONE (t=0.07s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=12.10s).
Accumulating evaluation results...
DONE (t=3.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.025
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.023
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.015
 Average Recall     (AR) @[ IoU=0.5

0.015437381136697206

In [ ]:
od = torch.load(args.checkpoint)

In [16]:
for nbatch, (img, img_id, img_size, _, _) in enumerate(val_dataloader):
    with torch.no_grad():
        img = img.cuda()
        out = ssd_model(img)
        

KeyboardInterrupt: 

In [ ]:
def eval_model(data_loader, model, device='cpu', print_freq=10):
    # print('Evaluation model ', model.arch)
    
    criterion = torch.nn.CrossEntropyLoss().to(device)
    
    loss = tnt.meter.AverageValueMeter()
    classerr = tnt.meter.ClassErrorMeter(accuracy=True, topk=(1, 5))
    # apmeter = tnt.meter.APMeter()

    total_samples = len(data_loader.sampler)
    batch_size = data_loader.batch_size
    total_steps = math.ceil(total_samples / batch_size)
    print('{0} samples ({1} per mini-batch)'.format(total_samples, batch_size))

    # Switch to evaluation mode
    model.eval()

    for step, (inputs, target) in enumerate(data_loader):
        with torch.no_grad():
            inputs, target = inputs.to(device), target.to(device)
            # compute output from model
            output = model(inputs)
            target = target_labels(dataset,target).to(device)
            # compute loss and measure accuracy
            loss.add(criterion(output, target).item())
            classerr.add(output.data, target)

            if (step + 1) % print_freq == 0:
                print('[{:3d}/{:3d}] Top1: {:.3f}  Top5: {:.3f}  Loss: {:.3f}'.format(
                      step + 1, total_steps, classerr.value(1), classerr.value(5), loss.mean), flush=True)
    print('----------')
    print('Overall ==> Top1: {:.3f}  Top5: {:.3f}  Loss: {:.3f}  PPL: {:.3f}'.format(
        classerr.value(1), classerr.value(5), loss.mean, np.exp(loss.mean)), flush=True)

In [ ]:
# def train(train_loop_func, logger, args):
#     # Check that GPUs are actually available
#     use_cuda = not args.no_cuda
# 
#     # Setup multi-GPU if necessary
#     args.distributed = False
#     if 'WORLD_SIZE' in os.environ:
#         args.distributed = int(os.environ['WORLD_SIZE']) > 1
# 
#     if args.distributed:
#         torch.cuda.set_device(args.local_rank)
#         torch.distributed.init_process_group(backend='nccl', init_method='env://')
#         args.N_gpu = torch.distributed.get_world_size()
#     else:
#         args.N_gpu = 1
# 
#     if args.seed is None:
#         args.seed = np.random.randint(1e4)
# 
#     if args.distributed:
#         args.seed = (args.seed + torch.distributed.get_rank()) % 2**32
#     print("Using seed = {}".format(args.seed))
#     torch.manual_seed(args.seed)
#     np.random.seed(seed=args.seed)
# 
# 
#     # Setup data, defaults
#     dboxes = dboxes300_coco()
#     encoder = Encoder(dboxes)
#     cocoGt = get_coco_ground_truth(args)
# 
#     # train_loader = get_train_loader(args, args.seed - 2**31)
# 
#     val_dataset = get_val_dataset(args)
#     val_dataloader = get_val_dataloader(val_dataset, args)
# 
#     ssd300 = SSD300(backbone=ResNet(args.backbone, args.backbone_path))
#     args.learning_rate = args.learning_rate * args.N_gpu * (args.batch_size / 32)
#     start_epoch = 0
#     iteration = 0
#     loss_func = Loss(dboxes)
# 
#     if use_cuda:
#         ssd300.cuda()
#         loss_func.cuda()
# 
#     optimizer = torch.optim.SGD(tencent_trick(ssd300), lr=args.learning_rate,
#                                     momentum=args.momentum, weight_decay=args.weight_decay)
#     scheduler = MultiStepLR(optimizer=optimizer, milestones=args.multistep, gamma=0.1)
#     if args.amp:
#         ssd300, optimizer = amp.initialize(ssd300, optimizer, opt_level='O2')
# 
#     if args.distributed:
#         ssd300 = DDP(ssd300)
# 
#     if args.checkpoint is not None:
#         if os.path.isfile(args.checkpoint):
#             load_checkpoint(ssd300.module if args.distributed else ssd300, args.checkpoint)
#             checkpoint = torch.load(args.checkpoint,
#                                     map_location=lambda storage, loc: storage.cuda(torch.cuda.current_device()))
#             start_epoch = checkpoint['epoch']
#             iteration = checkpoint['iteration']
#             scheduler.load_state_dict(checkpoint['scheduler'])
#             optimizer.load_state_dict(checkpoint['optimizer'])
#         else:
#             print('Provided checkpoint is not path to a file')
#             return
# 
#     inv_map = {v: k for k, v in val_dataset.label_map.items()}
# 
#     total_time = 0
# 
#     if args.mode == 'evaluation':
#         acc = evaluate(ssd300, val_dataloader, cocoGt, encoder, inv_map, args)
#         if args.local_rank == 0:
#             print('Model precision {} mAP'.format(acc))
# 
#         return


